# Data Understanding

In [1]:
%matplotlib inline
#Initialise Libraries
import pandas as pd
import sklearn
import numpy as np
import matplotlib as mpl


In [2]:
#import the initial training data
training = pd.read_csv('../DATA/cs-training.csv',index_col= 'idx' )

In [3]:
training.head(n=5)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30to59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60to89DaysPastDueNotWorse,NumberOfDependents
idx,,,,,,,,,,,
1,1,0.766127,45,2,0.802982,9120,13,0,6,0,2
2,0,0.957151,40,0,0.121876,2600,4,0,0,0,1
3,0,0.658180,38,1,0.085113,3042,2,1,0,0,0
4,0,0.233810,30,0,0.036050,3300,5,0,0,0,0
5,0,0.907239,49,1,0.024926,63588,7,0,1,0,0


### Feature Discussion

#### The training dataset has 150,000 observations, with the target being an indicator (i.e Y/N) to the individual experiencing 90 days past due delinquency or worse. 

** The features we will seek to make use of are:** 
> **RevolvingUtilizationOfUnsecuredLines** = Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits. (percentage)

> **age** = The age of borrower in years. (integer)

> **NumberOfTime30-59DaysPastDueNotWorse** = Number of times borrower has been 30-59 days past due but no worse in the last 2 years. (integer)

> **DebtRatio** = Monthly debt payments, alimony,living costs divided by monthy gross income. (percentage)

> **MonthlyIncome** = Monthly income (real)

> **NumberOfOpenCreditLinesAndLoans** = Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards). (integer)

> **NumberOfTimes90DaysLate** = Number of times borrower has been 90 days or more past due. (integer)

> **NumberRealEstateLoansOrLines** = Number of mortgage and real estate loans including home equity lines of credit. (integer)

> **NumberOfTime60-89DaysPastDueNotWorse** = Number of times borrower has been 60-89 days past due but no worse in the last 2 years. (integer) 

> **NumberOfDependents** = Number of dependents in family excluding themselves (spouse, children etc.). (integer)




In [4]:
#Lets get a feel for all the columns in the dataframe
training.describe(include='all')

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30to59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60to89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,120269.000000,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6670.221237,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,14384.674215,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3400.000000,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5400.000000,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8249.000000,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3008750.000000,58.000000,98.000000,54.000000,98.000000,20.000000


In [5]:
# Count the number and percentage of Serious Deliquencies in the training set
print "The number of Serious Deliquencies in the training set is" , training.SeriousDlqin2yrs.sum()
print "The percentage of deliquencies in the training set is ", \
float(sum(training.SeriousDlqin2yrs == 1) * 100) / float(training.SeriousDlqin2yrs.count()) , "%" 

The number of Serious Deliquencies in the training set is 10026
The percentage of deliquencies in the training set is  6.684 %


#### Observations:
> So the training data is quite unbalanced. This is to a large extent expected given the nature of the problem. This will cause issues even with some very good learning algorithms unless we carefully control for this imbalance. If we dont, the learner will often consider this as noise . We may still end up with a high accuracy, but low sensitivity. Ultimately we want to be able to predict the number the individuals who will be under financial distress, as these are the applications which we are more likely to refuse as a bank, as this threatens the Banks' liquidity position.

In [6]:
#Check for missing values also. This will present an issue later when we attempt to applying modelling frameworks. 
#So we better check now.  
training.isnull().sum()

SeriousDlqin2yrs                             0
RevolvingUtilizationOfUnsecuredLines         0
age                                          0
NumberOfTime30to59DaysPastDueNotWorse        0
DebtRatio                                    0
MonthlyIncome                            29731
NumberOfOpenCreditLinesAndLoans              0
NumberOfTimes90DaysLate                      0
NumberRealEstateLoansOrLines                 0
NumberOfTime60to89DaysPastDueNotWorse        0
NumberOfDependents                        3924
dtype: int64

#### Observations:
> The training data has no missing values for almost all variables except for the monthly income and also the number of dependants. The monthly income variable will need to be treated with caution. Here we need to ask ourselves, what is the data generating process? The data would most likely be gathered from loan applications. This would presumably be a mandatory field. So there is data loss which we will need to account for. As one would expect monthly income to have a strong relationship with the rate of delinquency, we will need to infer the monthly income somehow. If we consider the variable 'Debt Ratio', this is "Monthly debt payments, alimony, living costs divided by monthy gross income". The data here is full, so we may be able to use this variable to impute values for monthly income, based on this ratio the like ratios of individuals and their reported incomes. The number of dependents also presents an issue for a learning algorithm. Using some common sense, we could argue that the field may not have been filled out because the individual may not have dependents. So it could be argued that ever missing value is "0". Before such a crude rule is applied, we may decide to cluster individuals to see if we can find commonalities. 

*** (nb). Some leaners can handle missing data. For instance, classification and regression trees can deal with missing data by using surrogate splits. However the scikit learn implementation does not support this so we will be explicity controlling for missing data in all models. ***

### Feature Analysis 

In [7]:
# Given all variables are numeric, we will look at the correlations in the data and histograms.
#training.corr(method='kendall')  #non-parametric method .. (nb) Takes longer to run
training.corr(method='pearson') #simple linear correlation

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30to59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60to89DaysPastDueNotWorse,NumberOfDependents
SeriousDlqin2yrs,1.000000,-0.001802,-0.115386,0.125587,-0.007602,-0.019746,-0.029669,0.117175,-0.007038,0.102261,0.046048
RevolvingUtilizationOfUnsecuredLines,-0.001802,1.000000,-0.005898,-0.001314,0.003961,0.007124,-0.011281,-0.001061,0.006235,-0.001048,0.001557
age,-0.115386,-0.005898,1.000000,-0.062995,0.024188,0.037717,0.147705,-0.061005,0.033150,-0.057159,-0.213303
NumberOfTime30to59DaysPastDueNotWorse,0.125587,-0.001314,-0.062995,1.000000,-0.006542,-0.010217,-0.055312,0.983603,-0.030565,0.987005,-0.002680
DebtRatio,-0.007602,0.003961,0.024188,-0.006542,1.000000,-0.028712,0.049565,-0.008320,0.120046,-0.007533,-0.040673
MonthlyIncome,-0.019746,0.007124,0.037717,-0.010217,-0.028712,1.000000,0.091455,-0.012743,0.124959,-0.011116,0.062647
NumberOfOpenCreditLinesAndLoans,-0.029669,-0.011281,0.147705,-0.055312,0.049565,0.091455,1.000000,-0.079984,0.433959,-0.071077,0.065322
NumberOfTimes90DaysLate,0.117175,-0.001061,-0.061005,0.983603,-0.008320,-0.012743,-0.079984,1.000000,-0.045205,0.992796,-0.010176
NumberRealEstateLoansOrLines,-0.007038,0.006235,0.033150,-0.030565,0.120046,0.124959,0.433959,-0.045205,1.000000,-0.039722,0.124684
NumberOfTime60to89DaysPastDueNotWorse,0.102261,-0.001048,-0.057159,0.987005,-0.007533,-0.011116,-0.071077,0.992796,-0.039722,1.000000,-0.010922


### Analyising the 1's

In [8]:
ones = training[training.SeriousDlqin2yrs == 1]

ones.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30to59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60to89DaysPastDueNotWorse,NumberOfDependents
count,10026,10026.000000,10026.000000,10026.00000,10026.000000,8357.000000,10026.000000,10026.000000,10026.000000,10026.000000,9847.000000
mean,1,4.367282,45.926591,2.38849,295.121066,5630.826493,7.882306,2.091362,0.988530,1.828047,0.948208
std,0,131.835778,12.916289,11.73451,1238.360283,6171.719674,5.653601,11.762760,1.425723,11.753068,1.219367
min,1,0.000000,21.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1,0.398219,36.000000,0.00000,0.193979,2963.000000,4.000000,0.000000,0.000000,0.000000,0.000000
50%,1,0.838853,45.000000,0.00000,0.428227,4500.000000,7.000000,0.000000,1.000000,0.000000,0.000000
75%,1,1.000000,54.000000,2.00000,0.892371,6800.000000,11.000000,1.000000,2.000000,1.000000,2.000000
max,1,8328.000000,101.000000,98.00000,38793.000000,250000.000000,57.000000,98.000000,29.000000,98.000000,8.000000
